Lasso on synthetic data :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

breakCheckValue = 1e-3


# Checks to see if values of w are changing enough to continue
# the coordinate Descend algorithm.
def breakCheck(w_1, w_2):
    if (np.any(np.abs(np.add(w_1, -w_2)) > breakCheckValue)):
        return True
    return False


# coordinate Descend
def coordDescend(X, Y, lambda_, w_initial):
    n_, d_ = np.shape(X)

    w = np.copy(w_initial)
    breakCondition = True
    
    # commented out since data is regularized
    # X_ = (X - train_mean) / train_sigma
    X_ = X

    b = np.mean(Y - np.matmul(w.reshape(1, d_), np.transpose(X_)))
    # sanity check
    oldVal = np.matmul((Y - np.matmul(w.reshape(1, d_), X_.T) - b),
                       (Y - np.matmul(w.reshape(1, d_), X_.T) - b).T) + lambda_ * np.sum(np.abs(w))

    a = np.empty(d_)
    for k in range(0, d_):
        a[k] = 2 * np.matmul(X_[:, k], X_[:, k])

    while (breakCondition):
        b = np.mean(Y - np.matmul(w.reshape(1, d_), np.transpose(X_)))
        w_old = np.copy(w)
        for k in range(0, d_):
            X_c = np.delete(X_, k, axis=1)
            w_c = np.delete(w, k)
            c_k = 2 * np.matmul(Y - b - np.matmul(X_c, w_c), X_[:, k])

            if c_k < -lambda_:
                w[k] = (c_k + lambda_) / a[k]
            elif c_k > lambda_:
                w[k] = (c_k - lambda_) / a[k]
            else:
                w[k] = 0
            # sanity check - checking that it is reducing all the time
        newVal = np.matmul((Y - np.matmul(w.reshape(1, d_), X_.T) - b),
                           (Y - np.matmul(w.reshape(1, d_), X_.T) - b).T) + lambda_ * np.sum(np.abs(w))
        if newVal > oldVal:
            print("Sanity Check Error!")
        oldVal = np.copy(newVal)
        breakCondition = breakCheck(w, w_old)
    print(lambda_)
    return w, b


def lambdaMax(X, Y):
    return np.max(2 * np.dot((Y - np.mean(Y)), X))


# testing solver with synthetic data
def Lasso_syn(X, Y):
    lambda_max = lambdaMax(X, Y)
    lambda_ = lambda_max
    n_, d_ = np.shape(X)
    w = np.zeros(d_)
    lambdas = []
    # a list containing all the w values calculated by lasso
    omegas = []
    non_zero = 0
    while non_zero <= (0.99) * d_:
        w_l, b = coordDescend(X, Y, lambda_, w)
        lambdas.append(lambda_)
        lambda_ /= 1.5
        omegas.append(np.copy(w_l))
        non_zero = np.count_nonzero(w_l)
        w = w_l
        print(non_zero)
    return lambdas, omegas


def plotNonZero(lambdas, omegas, title):
    nonZeros = []
    for w in omegas:
        nonZeros.append(np.count_nonzero(w))
    plt.plot(lambdas, nonZeros)
    plt.xscale('log')
    plt.xlabel('log(λ)')
    plt.ylabel('Non-Zeros')
    plt.title(title)
    plt.show()


def FdrTprPlot(omegas, k_, dim):
    d_ = dim
    fdr_list = []
    tpr_list = []
    for c in range(0, np.shape(omegas)[0]):
        p0 = omegas[c]
        p1 = omegas[c][0: (k_ - 1)]
        p2 = omegas[c][k_: (d_ - 1)]
        nonZeros = np.count_nonzero(p0)
        true_nonZeros = np.count_nonzero(p1)
        false_nonZeros = np.count_nonzero(p2)
        if nonZeros > 0:
            fdr_list.append(false_nonZeros / nonZeros)
            tpr_list.append(true_nonZeros / k_)

    plt.plot(fdr_list, tpr_list)
    plt.xlabel('FDR')
    plt.ylabel('TPR')
    plt.title('Plot 2 (A.4 b)')
    plt.show()
# creating synthetic data and plotting non-zeros
mu = 0
sigma = 1
n = 500
d = 1000
k = 100
X_s = np.random.normal(mu, sigma, d).reshape((1, d))
for i in range(1, n):
    x_i = np.random.normal(mu, sigma, d).reshape((1, d))
    X_s = np.append(X_s, x_i, axis=0)
w_ = np.zeros(d)
for j in range(0, k):
    w_[j] = j / k
w_ = w_.reshape((1, d))
e_ = np.random.normal(0, sigma ^ 2, n).reshape((1, n))

Y_s = np.matmul(w_, np.transpose(X_s)) + e_

# non-zeros vs (log) lambda
lambdas_, omegas_ = Lasso_syn(X_s, Y_s)
plotNonZero(lambdas_, omegas_, 'Plot 1 (A.4 a)')

# FDR/TPR plot
FdrTprPlot(omegas_, k, d)